In [ ]:
'''
    Notebook to develop/test algorithms for detecting bounding box collisions
'''

import sys
import math
import random
import copy
sys.path.append('../src/')
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.patches import Rectangle

In [ ]:


def check_collision(b1_ll, b1_ur, b2_ll, b2_ur):
    return isOverlapping([b1_ll[0], b1_ur[0]], [b2_ll[0], b2_ur[0]]) and isOverlapping([b1_ll[1], b1_ur[1]], [b2_ll[1], b2_ur[1]])
    
def isOverlapping(b1, b2):
    return ((b1[1] >= b2[0]) and (b2[1] >= b1[0]))

def plotBox(ll, ur, ax):
    ax.add_patch(Rectangle(tuple(ll), ur[0]-ll[0], ur[1]-ll[1]))

In [ ]:
b1_ll = [10, 10]
b1_ur = [20, 20]

b2_ll = [5, 12]
b2_ur = [25, 13]

plt.figure()
ax = plt.gca()
ax.plot([0, 1], [0, 1])
plotBox(b1_ll, b1_ur, ax)
ax.add_patch(Rectangle(tuple(b2_ll), b2_ur[0]-b2_ll[0], b2_ur[1]-b2_ll[1], facecolor= 'red'))

if check_collision(b1_ll, b1_ur, b2_ll, b2_ur):
    print("COLLISION DETECTED!")

In [ ]:
'''
    Test detection of colliding tiles with a certain bounding box
'''
tile_size = 512
n_x_tiles = 29
n_y_tiles = 100
map_origin = [1310910, 6067376]

map_size_x = n_x_tiles*tile_size
map_size_y = n_y_tiles*tile_size

def plotTile(tile_index, ax):
    tile_x0 = (tile_index % n_x_tiles) * tile_size
    tile_y0 = math.floor(tile_index / n_x_tiles) * tile_size
    ax.add_patch(
        Rectangle((tile_x0, tile_y0), tile_size, tile_size, 
            edgecolor = 'black',
            fill=False)
    )
    
'''
    Get all tile indices that collide with a box defined by a lower-left point (ll)
    and an upper-right point (ur). All coordinates in mercator [x, y]
'''
def get_colliding_tiles(ll, ur, tile_size, n_x_tiles, map_origin):
    colliding_tile_idx = []
    # Map origin is lower-left corner of the map.

    # Determin offsets from origin
    offset_ll = np.array(ll) - np.array(map_origin)
    offset_ur = np.array(ur) - np.array(map_origin)
    # Determine tile index for tile that includes ll point
    tile_idx_ll = n_x_tiles*math.floor(offset_ll[1] / tile_size) + math.floor(offset_ll[0] / tile_size)

    # Determine number of tiles in x and y direction that collide with box
    dx_tiles = math.floor(offset_ur[0] / tile_size) - math.floor(offset_ll[0] / tile_size) + 1
    dy_tiles = math.floor(offset_ur[1] / tile_size) - math.floor(offset_ll[1] / tile_size) + 1

    print(f"dx_tiles: {dx_tiles}, dy_tiles: {dy_tiles}")
    
    # Generate tile ids
    for i in range(dy_tiles):
        colliding_tile_idx.extend([j + tile_idx_ll + n_x_tiles*i for j in range(0, dx_tiles)])

    return colliding_tile_idx

print()

box_ll = [1324191, 6070282]
box_ur = [1324370, 6070294]

tile_idxes = get_colliding_tiles(box_ll, box_ur, tile_size, n_x_tiles, map_origin)

print(f"Found {len(tile_idxes)} colliding tiles")

plt.figure()
ax = plt.gca()
for idx in tile_idxes:
    plotTile(idx, ax)

plotBox(box_ll, box_ur, ax)
    
plt.plot([box_ll[0], box_ll[0]+1], [box_ll[1], box_ll[1]+1])
plt.xlim((box_ll[0] - tile_size-1, box_ur[0] + tile_size+1))
plt.ylim((box_ll[1] - tile_size-1, box_ur[1] + tile_size+1))
ax.set_aspect('equal', adjustable='box')
plt.draw()